In [1]:
from sqlalchemy import MetaData, create_engine, ForeignKey, Column, Integer, String, Float, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from datetime import datetime as dt
from sqlalchemy.sql import func

engine = create_engine('sqlite:///corona_analysis.db', echo=True)
meta_data = MetaData(bind=engine)
Base = declarative_base()

# Agegroups

In [2]:
class Agegroups_05y(Base):
   __tablename__ = '_agegroups_05y'

   agegroups_05y_id = Column(Integer, primary_key = True)
   agegroup = Column(String, nullable=False, unique=True)
   # meta cols
   created_on = Column(DateTime(timezone=True), nullable=False, default=func.now())
   updated_on = Column(DateTime(timezone=True), default=func.now())
   unique_key = Column(String, nullable=False, unique=True)

class Agegroups_10y(Base):
   __tablename__ = '_agegroups_10y'

   agegroups_10y_id = Column(Integer, primary_key = True)
   agegroup = Column(String, nullable=False, unique=True)
   number_observations = Column(Integer)
   avg_age = Column(Float)
   # meta cols
   created_on = Column(DateTime(timezone=True), nullable=False, default=func.now())
   updated_on = Column(DateTime(timezone=True), default=func.now())
   unique_key = Column(String, nullable=False, unique=True)

In [3]:
dt.today()

datetime.datetime(2022, 7, 3, 3, 47, 27, 176820)

In [4]:
def add_new_agegroup_05y(session: sessionmaker, agegroup: str):
    """
    Adds a new agegroup with a 5-year-interval
    """
    
    # check if agegroup already exist
    new_agegroup = (
        session.query(Agegroups_05y)
        .filter(Agegroups_05y.agegroup == agegroup)
    ).one_or_none()

    if new_agegroup is not None:
        return
    
    # create new agegroup
    new_agegroup = Agegroups_05y(
        agegroup=agegroup,
        unique_key=agegroup
    )
    
    # write to DB
    session.add(new_agegroup)
    session.commit()

In [5]:
Base.metadata.create_all(engine)

2022-07-03 03:47:29,711 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-03 03:47:29,712 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("_agegroups_05y")
2022-07-03 03:47:29,712 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-03 03:47:29,713 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("_agegroups_05y")
2022-07-03 03:47:29,713 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-03 03:47:29,714 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("_agegroups_10y")
2022-07-03 03:47:29,714 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-03 03:47:29,714 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("_agegroups_10y")
2022-07-03 03:47:29,715 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-03 03:47:29,716 INFO sqlalchemy.engine.Engine 
CREATE TABLE _agegroups_05y (
	agegroups_05y_id INTEGER NOT NULL, 
	agegroup VARCHAR NOT NULL, 
	created_on DATETIME NOT NULL, 
	updated_on DATETIME, 
	unique_key VARCHAR NOT NULL, 
	PRIMARY KEY (agegroups_05y_id), 
	UNIQUE (a

In [6]:
# create session object which will be used for connection
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [7]:
agegroups_05y = [
    "00-04",
    "05-09",
    "10-14",
    "15-19",
    "20-24",
    "25-29",
    "30-34",
    "35-39",
    "40-44",
    "45-49",
    "50-54",
    "55-59",
    "60-64",
    "65-69",
    "70-74",
    "75-79",
    "80+",
    "UNK"
]
for item in agegroups_05y:
    add_new_agegroup_05y(session=session, agegroup=item)

2022-07-03 03:47:31,429 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-03 03:47:31,431 INFO sqlalchemy.engine.Engine SELECT _agegroups_05y.agegroups_05y_id AS _agegroups_05y_agegroups_05y_id, _agegroups_05y.agegroup AS _agegroups_05y_agegroup, _agegroups_05y.created_on AS _agegroups_05y_created_on, _agegroups_05y.updated_on AS _agegroups_05y_updated_on, _agegroups_05y.unique_key AS _agegroups_05y_unique_key 
FROM _agegroups_05y 
WHERE _agegroups_05y.agegroup = ?
2022-07-03 03:47:31,431 INFO sqlalchemy.engine.Engine [generated in 0.00054s] ('00-04',)
2022-07-03 03:47:31,433 INFO sqlalchemy.engine.Engine INSERT INTO _agegroups_05y (agegroup, created_on, updated_on, unique_key) VALUES (?, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, ?)
2022-07-03 03:47:31,434 INFO sqlalchemy.engine.Engine [generated in 0.00060s] ('00-04', '00-04')
2022-07-03 03:47:31,435 INFO sqlalchemy.engine.Engine COMMIT
2022-07-03 03:47:31,436 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-03 03:47:31,437 INF

In [8]:
def add_new_agegroup_10y(session: sessionmaker, agegroup: str, number_observations: int, avg_age: float):
    """
    Adds a new agegroup with a 10-year-interval
    """
    
    # check if agegroup already exist
    new_agegroup = (
        session.query(Agegroups_10y)
        .filter(Agegroups_10y.agegroup == agegroup)
    ).one_or_none()

    if new_agegroup is not None:
        return
    
    # create new agegroup
    new_agegroup = Agegroups_10y(
        agegroup=agegroup, 
        number_observations=number_observations, 
        avg_age=avg_age,
        unique_key=agegroup
    )
    
    # write to DB
    session.add(new_agegroup)
    session.commit()

In [9]:
agegroups_10y = { # agegroup, [number_observations, avg_age]
    "00-09": [10, 4.5],
    "10-19": [10, 14.5],
    "20-29": [10, 24.5],
    "30-39": [10, 34.5],
    "40-49": [10, 44.5],
    "50-59": [10, 54.5],
    "60-69": [10, 64.5],
    "70-79": [10, 74.5],
    "80+": [21, 90],
    "UNK": [0, 0],
}
for k, v, in agegroups_10y.items():
    add_new_agegroup_10y(session=session, agegroup=k, number_observations=v[0], avg_age=v[1])

2022-07-03 03:47:33,626 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-03 03:47:33,628 INFO sqlalchemy.engine.Engine SELECT _agegroups_10y.agegroups_10y_id AS _agegroups_10y_agegroups_10y_id, _agegroups_10y.agegroup AS _agegroups_10y_agegroup, _agegroups_10y.number_observations AS _agegroups_10y_number_observations, _agegroups_10y.avg_age AS _agegroups_10y_avg_age, _agegroups_10y.created_on AS _agegroups_10y_created_on, _agegroups_10y.updated_on AS _agegroups_10y_updated_on, _agegroups_10y.unique_key AS _agegroups_10y_unique_key 
FROM _agegroups_10y 
WHERE _agegroups_10y.agegroup = ?
2022-07-03 03:47:33,628 INFO sqlalchemy.engine.Engine [generated in 0.00050s] ('00-09',)
2022-07-03 03:47:33,629 INFO sqlalchemy.engine.Engine INSERT INTO _agegroups_10y (agegroup, number_observations, avg_age, created_on, updated_on, unique_key) VALUES (?, ?, ?, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP, ?)
2022-07-03 03:47:33,630 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ('00-09', 10, 4.5,

In [10]:
def get_agegroups_05y(session: sessionmaker):
    """Get a list of agegroup objects with a 05-year interval"""
    return session.query(Agegroups_05y).order_by(Agegroups_05y.agegroup).all()

def get_agegroups_10y(session: sessionmaker):
    """Get a list of agegroup objects with a 10-year interval"""
    return session.query(Agegroups_10y).order_by(Agegroups_10y.agegroup).all()

# Calendar

In [17]:
engine = create_engine('sqlite:///corona_analysis.db', echo=True)
meta_data = MetaData(bind=engine)
Base = declarative_base()

In [18]:
class CalendarYears(Base):
    __tablename__ = "_calendar_years"

    calendar_years_id = Column(Integer, primary_key=True)
    iso_year = Column(Integer, nullable=False, unique=True)
    # meta cols
    created_on = Column(DateTime(timezone=True), nullable=False, default=func.now())
    updated_on = Column(DateTime(timezone=True), default=func.now())
    unique_key = Column(String, nullable=False, unique=True)
    # relationships
    calendar_weeks = relationship("CalendarWeeks", backref="_calendar_years", cascade="all, delete")

In [19]:
CalendarYears.calendar_years_id.name

'calendar_years_id'

In [20]:
class CalendarWeeks(Base):
    __tablename__ = "_calendar_weeks"

    calendar_weeks_id = Column(Integer, primary_key=True)
    calendar_years_fk = Column(Integer, ForeignKey("_calendar_years.calendar_years_id", ondelete="CASCADE", onupdate="CASCADE"), nullable=False)
    iso_week = Column(Integer, nullable=False)
    iso_key = Column(Integer, nullable=False, unique=True)
    # meta cols
    created_on = Column(DateTime(timezone=True), nullable=False, default=func.now())
    updated_on = Column(DateTime(timezone=True), default=func.now())
    unique_key = Column(String, nullable=False, unique=True)
    # relationships
    calendar_years = relationship("CalendarDays", backref="_calendar_weeks", cascade="all, delete")

In [21]:
class CalendarDays(Base):
    __tablename__ = "_calendar_days"

    calendar_days_id = Column(Integer, primary_key=True)
    calendar_weeks_fk = Column(Integer, ForeignKey("_calendar_weeks.calendar_weeks_id", ondelete="CASCADE", onupdate="CASCADE"), nullable=False)
    iso_day = Column(String, nullable=False, unique=True)
    # meta cols
    created_on = Column(DateTime(timezone=True), nullable=False, default=func.now())
    updated_on = Column(DateTime(timezone=True), default=func.now())
    unique_key = Column(String, nullable=False, unique=True)

In [22]:
Base.metadata.create_all(engine)

2022-07-03 03:49:00,799 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-03 03:49:00,801 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("_calendar_years")
2022-07-03 03:49:00,801 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-03 03:49:00,802 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("_calendar_weeks")
2022-07-03 03:49:00,802 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-03 03:49:00,803 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("_calendar_days")
2022-07-03 03:49:00,803 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-03 03:49:00,803 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("_calendar_days")
2022-07-03 03:49:00,804 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-03 03:49:00,805 INFO sqlalchemy.engine.Engine 
CREATE TABLE _calendar_days (
	calendar_days_id INTEGER NOT NULL, 
	calendar_weeks_fk INTEGER NOT NULL, 
	iso_day VARCHAR NOT NULL, 
	created_on DATETIME NOT NULL, 
	updated_on DATETIME, 
	unique_key VARCHAR NOT NULL, 
	PRIM

In [36]:
from sqlalchemy.engine.row import Row

def get_calendar_year(session: sessionmaker, calendar_year: int) -> Row:
    return (
        session.query(CalendarYears.iso_year)
        .filter(CalendarYears.iso_year == calendar_year)
    ).one_or_none()

In [37]:
def get_calendar_years(session: sessionmaker):
    """Get a list of calendar years objects"""
    return session.query(CalendarYears).order_by(CalendarYears.iso_year).all()

# Map Years with its Foreign Keys
calendar_years = get_calendar_years(session=session)
calendar_years_dict = {}
calendar_years_dict_reverse = {}
for year in calendar_years:
    calendar_years_dict[year.iso_year] = year.calendar_years_id
for year in calendar_years:
    calendar_years_dict_reverse[year.calendar_years_id] = year.iso_year
calendar_years_dict

2022-07-03 03:51:13,975 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-07-03 03:51:13,976 INFO sqlalchemy.engine.Engine SELECT _calendar_years.calendar_years_id AS _calendar_years_calendar_years_id, _calendar_years.iso_year AS _calendar_years_iso_year, _calendar_years.created_on AS _calendar_years_created_on, _calendar_years.updated_on AS _calendar_years_updated_on, _calendar_years.unique_key AS _calendar_years_unique_key 
FROM _calendar_years ORDER BY _calendar_years.iso_year
2022-07-03 03:51:13,977 INFO sqlalchemy.engine.Engine [cached since 109.6s ago] ()


{1990: 1,
 1991: 2,
 1992: 3,
 1993: 4,
 1994: 5,
 1995: 6,
 1996: 7,
 1997: 8,
 1998: 9,
 1999: 10,
 2000: 11,
 2001: 12,
 2002: 13,
 2003: 14,
 2004: 15,
 2005: 16,
 2006: 17,
 2007: 18,
 2008: 19,
 2009: 20,
 2010: 21,
 2011: 22,
 2012: 23,
 2013: 24,
 2014: 25,
 2015: 26,
 2016: 27,
 2017: 28,
 2018: 29,
 2019: 30,
 2020: 31,
 2021: 32,
 2022: 33,
 2023: 34,
 2024: 35,
 2025: 36,
 2026: 37,
 2027: 38,
 2028: 39,
 2029: 40,
 2030: 41,
 2031: 42,
 2032: 43,
 2033: 44,
 2034: 45,
 2035: 46,
 2036: 47,
 2037: 48,
 2038: 49,
 2039: 50,
 2040: 51,
 2041: 52,
 2042: 53,
 2043: 54,
 2044: 55,
 2045: 56,
 2046: 57,
 2047: 58,
 2048: 59,
 2049: 60}

In [51]:
def get_calendar_weeks(session: sessionmaker):
    """Get a list of calendar years objects"""
    return session.query(CalendarWeeks).order_by(CalendarWeeks.iso_key).all()

# Map Weeks with its Foreign Keys
calendar_weeks = get_calendar_weeks(session=session)
calendar_weeks_dict = {}
calendar_weeks_dict_reverse = {}
for week in calendar_weeks:
    calendar_weeks_dict[week.iso_key] = week.calendar_weeks_id
calendar_weeks_dict

2022-07-03 03:59:11,129 INFO sqlalchemy.engine.Engine SELECT _calendar_weeks.calendar_weeks_id AS _calendar_weeks_calendar_weeks_id, _calendar_weeks.calendar_years_fk AS _calendar_weeks_calendar_years_fk, _calendar_weeks.iso_week AS _calendar_weeks_iso_week, _calendar_weeks.iso_key AS _calendar_weeks_iso_key, _calendar_weeks.created_on AS _calendar_weeks_created_on, _calendar_weeks.updated_on AS _calendar_weeks_updated_on, _calendar_weeks.unique_key AS _calendar_weeks_unique_key 
FROM _calendar_weeks ORDER BY _calendar_weeks.iso_key
2022-07-03 03:59:11,130 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ()


{199001: 1,
 199002: 2,
 199003: 3,
 199004: 4,
 199005: 5,
 199006: 6,
 199007: 7,
 199008: 8,
 199009: 9,
 199010: 10,
 199011: 11,
 199012: 12,
 199013: 13,
 199014: 14,
 199015: 15,
 199016: 16,
 199017: 17,
 199018: 18,
 199019: 19,
 199020: 20,
 199021: 21,
 199022: 22,
 199023: 23,
 199024: 24,
 199025: 25,
 199026: 26,
 199027: 27,
 199028: 28,
 199029: 29,
 199030: 30,
 199031: 31,
 199032: 32,
 199033: 33,
 199034: 34,
 199035: 35,
 199036: 36,
 199037: 37,
 199038: 38,
 199039: 39,
 199040: 40,
 199041: 41,
 199042: 42,
 199043: 43,
 199044: 44,
 199045: 45,
 199046: 46,
 199047: 47,
 199048: 48,
 199049: 49,
 199050: 50,
 199051: 51,
 199052: 52,
 199101: 53,
 199102: 54,
 199103: 55,
 199104: 56,
 199105: 57,
 199106: 58,
 199107: 59,
 199108: 60,
 199109: 61,
 199110: 62,
 199111: 63,
 199112: 64,
 199113: 65,
 199114: 66,
 199115: 67,
 199116: 68,
 199117: 69,
 199118: 70,
 199119: 71,
 199120: 72,
 199121: 73,
 199122: 74,
 199123: 75,
 199124: 76,
 199125: 77,
 199126:

In [38]:
def add_new_calendar_year(session: sessionmaker, calendar_year: int) -> None:
    """
    Adds a new calendar year
    """
    
    # check if calendar year already exist
    year_exists = get_calendar_year(session=session, calendar_year=calendar_year)
    if year_exists is not None:
        return
   
    # create new entry
    new_calendar_year = CalendarYears(
        iso_year=calendar_year,
        unique_key=str(calendar_year)
    )
    
    # write to DB
    session.add(new_calendar_year)
    session.commit()

In [39]:
for year in range(1990, 2050):
    add_new_calendar_year(session=session, calendar_year=year)

2022-07-03 03:51:22,984 INFO sqlalchemy.engine.Engine SELECT _calendar_years.iso_year AS _calendar_years_iso_year 
FROM _calendar_years 
WHERE _calendar_years.iso_year = ?
2022-07-03 03:51:22,984 INFO sqlalchemy.engine.Engine [cached since 88.44s ago] (1990,)
2022-07-03 03:51:22,985 INFO sqlalchemy.engine.Engine SELECT _calendar_years.iso_year AS _calendar_years_iso_year 
FROM _calendar_years 
WHERE _calendar_years.iso_year = ?
2022-07-03 03:51:22,986 INFO sqlalchemy.engine.Engine [cached since 88.44s ago] (1991,)
2022-07-03 03:51:22,986 INFO sqlalchemy.engine.Engine SELECT _calendar_years.iso_year AS _calendar_years_iso_year 
FROM _calendar_years 
WHERE _calendar_years.iso_year = ?
2022-07-03 03:51:22,987 INFO sqlalchemy.engine.Engine [cached since 88.44s ago] (1992,)
2022-07-03 03:51:22,987 INFO sqlalchemy.engine.Engine SELECT _calendar_years.iso_year AS _calendar_years_iso_year 
FROM _calendar_years 
WHERE _calendar_years.iso_year = ?
2022-07-03 03:51:22,988 INFO sqlalchemy.engine.E

## ISO

In [40]:
import isoweek

calendar_weeks_dict = {}

for year, calendar_years_fk in calendar_years_dict.items():
    weeks_list = []
    weeks = isoweek.Week.last_week_of_year(year).week
    for week in range(1, weeks+1):
        weeks_list.append(week)
    calendar_weeks_dict[calendar_years_fk] = weeks_list

calendar_weeks_dict

{1: [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52],
 2: [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52],
 3: [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53],
 4: [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
 

In [105]:
isoweek.Week.last_week_of_year(1993).week

52

In [187]:
from calendar import Calendar
import pandas as pd
from datetime import date

daterange = pd.date_range(date(1990,1,1), date(2050,12,31))
l = {}
for single_date in  daterange:
    l[single_date.strftime("%G-%m-%d")] = single_date.strftime("%V")

df_date = pd.DataFrame.from_dict(l, orient="index").reset_index(level=0)
df_date.columns = ["iso_day", "iso_week"]
df_date.index += 1
df_date["iso_day"] = pd.to_datetime(df_date["iso_day"])
df_date["iso_year"] = pd.to_datetime(df_date["iso_day"]).dt.year
df_date["iso_key"] = df_date["iso_year"].astype(str) + df_date["iso_week"]
df_date

,iso_day,iso_week,iso_year,iso_key
1,1990-01-01,01,1990,199001
2,1990-01-02,01,1990,199001
3,1990-01-03,01,1990,199001
4,1990-01-04,01,1990,199001
5,1990-01-05,01,1990,199001
...,...,...,...,...
22215,2050-12-27,52,2050,205052
22216,2050-12-28,52,2050,205052
22217,2050-12-29,52,2050,205052
22218,2050-12-30,52,2050,205052


In [201]:
df_years = pd.DataFrame(df_date["iso_year"].unique(), columns=["iso_year"])
df_years.index += 1
df_years

,iso_year
1,1990
2,1991
3,1992
4,1993
5,1994
...,...
57,2046
58,2047
59,2048
60,2049


In [264]:
#calendar_years_fk = df_years.T.to_dict('records')
#calendar_years_fk_dict, *_ = calendar_years_fk # unpack
#calendar_years_fk_dict
tmp = df_years.copy()
tmp.reset_index(level=0, inplace=True)
calendar_years_fk = tmp.set_index("iso_year").T.to_dict('records')
calendar_years_fk_dict, *_ = calendar_years_fk # unpack
calendar_years_fk_dict


{1990: 1,
 1991: 2,
 1992: 3,
 1993: 4,
 1994: 5,
 1995: 6,
 1996: 7,
 1997: 8,
 1998: 9,
 1999: 10,
 2000: 11,
 2001: 12,
 2002: 13,
 2003: 14,
 2004: 15,
 2005: 16,
 2006: 17,
 2007: 18,
 2008: 19,
 2009: 20,
 2010: 21,
 2011: 22,
 2012: 23,
 2013: 24,
 2014: 25,
 2015: 26,
 2016: 27,
 2017: 28,
 2018: 29,
 2019: 30,
 2020: 31,
 2021: 32,
 2022: 33,
 2023: 34,
 2024: 35,
 2025: 36,
 2026: 37,
 2027: 38,
 2028: 39,
 2029: 40,
 2030: 41,
 2031: 42,
 2032: 43,
 2033: 44,
 2034: 45,
 2035: 46,
 2036: 47,
 2037: 48,
 2038: 49,
 2039: 50,
 2040: 51,
 2041: 52,
 2042: 53,
 2043: 54,
 2044: 55,
 2045: 56,
 2046: 57,
 2047: 58,
 2048: 59,
 2049: 60,
 2050: 61}

In [265]:
#df_weeks = df_date.groupby(["iso_key", "iso_week", "year"], as_index=False).sum()
df_weeks = df_date.groupby(['iso_key'], as_index=False).agg({'iso_day': 'first', 'iso_week': 'first', 'iso_year': 'first'})
df_weeks.drop("iso_day", axis=1, inplace=True)
df_weeks["iso_week"] = df_weeks["iso_week"].astype(int)
df_weeks["iso_year"] = df_weeks["iso_year"].astype(int)
#df_weeks["calendar_years_fk"] = df_weeks["iso_year"].map(df_years.to_dict())
df_weeks.index += 1
#d = {df_years.loc[index, 'iso_year']:index for index in df_years.index}
df_weeks["calendar_years_fk"] = df_weeks["iso_year"].astype(int).map(calendar_years_fk_dict)
df_weeks

,iso_key,iso_week,iso_year,calendar_years_fk
1,199001,1,1990,1
2,199002,2,1990,1
3,199003,3,1990,1
4,199004,4,1990,1
5,199005,5,1990,1
...,...,...,...,...
3179,205048,48,2050,61
3180,205049,49,2050,61
3181,205050,50,2050,61
3182,205051,51,2050,61


In [41]:
import pandas as pd
import numpy as np
# create dataframe
df = pd.DataFrame.from_dict(calendar_weeks_dict, orient="index")
df = df.T.unstack().to_frame().dropna().reset_index(level=0).applymap(np.int64)
df.columns = ["calendar_years_fk", "iso_week"]
df

,calendar_years_fk,iso_week
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
...,...,...
47,60,48
48,60,49
49,60,50
50,60,51


In [45]:
df["iso_key"] = df["calendar_years_fk"].map(calendar_years_dict_reverse).astype(str) + df["iso_week"].astype(str).str.zfill(2)
df["unique_key"] = df["iso_key"]
from datetime import datetime as dt
df["created_on"] = dt.now()
df["updated_on"] = dt.now()
df

,calendar_years_fk,iso_week,iso_key,unique_key,created_on,updated_on
0,1,1,199001,199001,2022-07-03 03:52:40.078135,2022-07-03 03:52:40.078563
1,1,2,199002,199002,2022-07-03 03:52:40.078135,2022-07-03 03:52:40.078563
2,1,3,199003,199003,2022-07-03 03:52:40.078135,2022-07-03 03:52:40.078563
3,1,4,199004,199004,2022-07-03 03:52:40.078135,2022-07-03 03:52:40.078563
4,1,5,199005,199005,2022-07-03 03:52:40.078135,2022-07-03 03:52:40.078563
...,...,...,...,...,...,...
47,60,48,204948,204948,2022-07-03 03:52:40.078135,2022-07-03 03:52:40.078563
48,60,49,204949,204949,2022-07-03 03:52:40.078135,2022-07-03 03:52:40.078563
49,60,50,204950,204950,2022-07-03 03:52:40.078135,2022-07-03 03:52:40.078563
50,60,51,204951,204951,2022-07-03 03:52:40.078135,2022-07-03 03:52:40.078563


In [157]:
def add_new_calendar_weeks(session: sessionmaker, calendar_year: int) -> None:
    """
    Adds calendar weeks for a given year by ISO standard
    """
    
    # check if calendar year already exist
    new_calendar_week = (
        session.query(CalendarWeeks)
        .filter(CalendarWeeks.iso_key == iso_key)
    ).one_or_none()

    if new_calendar_week is not None:
        return
    
    # create new agegroup
    new_calendar_week = CalendarWeeks(
        calendar_years_fk=calendar_years_fk,
        iso_week=iso_week,
        iso_key=iso_key,
        unique_key=iso_key
    )
    
    # write to DB
    session.add(new_calendar_week)
    session.commit()

In [46]:
from sqlalchemy.exc import NoSuchTableError
from sqlalchemy import Table

def checkIfTableExist(table_name: str):
    metadata = MetaData(engine)
    try:
        table = Table(table_name, metadata, autoload=True)
        return True
    except NoSuchTableError:
        return False
l = checkIfTableExist("_calendar_years")
print(l)

def getTableByName(table_name: str):
    metadata = MetaData(engine)
    try:
        table = Table(table_name, metadata, autoload=True)
        return table
    except NoSuchTableError:
        return None
l = getTableByName("_calendar_years")
print(l)

2022-07-03 03:52:45,881 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("_calendar_years")
2022-07-03 03:52:45,882 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-03 03:52:45,883 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-07-03 03:52:45,884 INFO sqlalchemy.engine.Engine [raw sql] ('_calendar_years',)
2022-07-03 03:52:45,884 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("_calendar_years")
2022-07-03 03:52:45,885 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-03 03:52:45,885 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("_calendar_years")
2022-07-03 03:52:45,885 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-03 03:52:45,886 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-07-03 03:52:45,886 INFO sqlalchemy.engine.Engine

In [47]:
import uuid
import sqlalchemy

# Credit to https://stackoverflow.com/questions/61366664/how-to-upsert-pandas-dataframe-to-postgresql-table
def upsert_df(df: pd.DataFrame, table_name: str, engine: sqlalchemy.engine.Engine) -> bool:

    # df must contain a unique_key column
    if "unique_key" not in df.columns:
        raise RuntimeError("DataFrame must contain a 'unique_key' column")

    # check if table exist. If not, create it using to_sql
    table_exist = checkIfTableExist(table_name=table_name)
    if not table_exist:
        df.to_sql(table_name, engine)
        return True

    # table exist, so use UPSERT logic...
    
    # 1. create temporary table with unique id
    tmp_table = f"tmp_{uuid.uuid4().hex[:6]}"
    df.to_sql(tmp_table, engine, index=True)

    # 2. 
    columns = list(df.columns)
    columns_str = ", ".join(col for col in columns)

    # The "excluded." prefix causes the column to refer to the value that 
    # would have been inserted if there been no conflict.
    update_columns_str = ", ".join(
        f'{col} = excluded.{col}' for col in columns
    )

    # 3.
    """ example:
    INSERT INTO _calendar_years(iso_year)
    VALUES(2050)
    ON CONFLICT(iso_year) DO UPDATE SET
    iso_year=excluded.iso_year;
    """
    query_upsert = f"""
        INSERT INTO {table_name}({columns_str})
        SELECT {columns_str} FROM {tmp_table} WHERE true
        ON CONFLICT(unique_key) DO UPDATE SET
        {update_columns_str};
    """

    # 4. execute upsert query & drop temporary table
    engine.execute(query_upsert)
    engine.execute(f"DROP TABLE {tmp_table}")

    return True

upsert_df(df, "_calendar_weeks", engine)


2022-07-03 03:52:46,467 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("_calendar_weeks")
2022-07-03 03:52:46,467 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-03 03:52:46,468 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-07-03 03:52:46,469 INFO sqlalchemy.engine.Engine [raw sql] ('_calendar_weeks',)
2022-07-03 03:52:46,470 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("_calendar_weeks")
2022-07-03 03:52:46,470 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-07-03 03:52:46,471 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-07-03 03:52:46,471 INFO sqlalchemy.engine.Engine [raw sql] ('_calendar_weeks',)
2022-07-03 03:52:46,472 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("_calendar_years")
2022-07-03 03:52:46,472 INFO sqlalchemy.

True

In [308]:
daterange = pd.date_range(date(1990,1,1), date(2050,12,31))
#len(daterange.to_list())
d = daterange.to_list()

t = dict()
for i in d:
    iso = i.strftime("%Y-%m-%d")
    week = i.strftime("%V")
    t[iso] = week
print(len(t))
dff = pd.DataFrame.from_dict(t, orient="index").reset_index(level=0)
dff.columns = ["iso_day", "iso_week"]
dff.index += 1
dff["iso_day"] = pd.to_datetime(dff["iso_day"])
dff["iso_year"] = pd.to_datetime(dff["iso_day"]).dt.year
dff["iso_key"] = dff["iso_year"].astype(str) + dff["iso_week"]
dff.to_csv("test3.csv", sep=";")
#from collections import Counter
#a = dict(Counter(t))
#len(a)

22280


In [289]:
d = dict()
l = daterange.to_list()
for single_date in l:
    iso_year = single_date.strftime("%G-%m-%d")
    iso_week = single_date.strftime("%V")
    d[iso_year] = iso_week
len(d)

TypeError: 'set' object does not support item assignment

In [2]:
import pandas as pd
from datetime import datetime, date

In [7]:
daterange = pd.date_range(date(1990, 1, 1), date(2050, 12, 31))
regular_days = list()
iso_weeks = list()
iso_years = list()
for s in daterange:
    regular_days.append(s.strftime("%Y-%m-%d"))
    iso_weeks.append(s.strftime("%V"))
    iso_years.append(s.strftime("%G"))
pd.DataFrame(regular_days).to_csv("regular_days.csv", sep=";")
pd.DataFrame(iso_weeks).to_csv("iso_weeks.csv", sep=";")
pd.DataFrame(iso_years).to_csv("iso_years.csv", sep=";")

In [9]:
import isoweek

isoweek.Week.last_week_of_year(1993)

isoweek.Week(1993, 52)

In [52]:
import pandas as pd
from datetime import datetime, date

daterange = pd.date_range(date(1990, 1, 1), date(2050, 12, 31))
iso = dict()

for s in daterange:
    l = list()
    iso_day = s.strftime("%Y-%m-%d")
    iso_year = s.isocalendar().year
    iso_week = s.isocalendar().week

    l.append(iso_year)
    l.append(iso_week)

    iso[iso_day] = l

# day df
df_day = pd.DataFrame.from_dict(iso, orient="index").reset_index(level=0)
df_day.columns = ["iso_day", "iso_year", "iso_week"]
df_day["iso_week"] = df_day["iso_week"].astype(str).str.zfill(2)
df_day["iso_key"] = df_day["iso_year"].astype(str) + df_day["iso_week"].astype(str)
df_day.index += 1

# week df
df_week = df_day.groupby(
    ["iso_key"], as_index=False
    ).agg({"iso_day": "first", "iso_year": "first", "iso_week": "first"}).copy()
df_week.drop("iso_day", axis=1, inplace=True)
df_week.index += 1

# year df
df_year = df_week.groupby(
    ["iso_year"], as_index=False
    ).agg({"iso_key": "first", "iso_week": "first"}).copy()
df_year.drop(["iso_key", "iso_week"], axis=1, inplace=True)
df_year.index = df_year.index.set_names(["ID"])
df_year.index += 1
df_year = df_year.reset_index().rename(columns={df_year.index.name: "calendar_years_id"})


# merge FK week
df_week = df_week.merge(df_year, how="left", left_on="iso_year", right_on="iso_year")
df_week.index = df_week.index.set_names(["ID"])
df_week.index += 1
df_week = df_week.reset_index().rename(columns={df_week.index.name: "calendar_weeks_id"})
df_week.rename(columns={"calendar_years_id": "calendar_years_fk"}, inplace=True)
df_week.drop("iso_year", axis=1, inplace=True)
df_week["iso_week"] = df_week["iso_week"].astype(int)
df_week

# merge FK day
df_day = df_day.merge(df_week, how="left", left_on="iso_key", right_on="iso_key", suffixes=('', '_y'))
df_day.drop(df_day.filter(regex="_y$").columns, axis=1, inplace=True)
df_day.drop(["iso_year", "iso_week", "iso_key", "calendar_years_fk"], axis=1, inplace=True)
df_day.rename(columns={"calendar_weeks_id": "calendar_weeks_fk"}, inplace=True)


In [53]:
df_year

,calendar_years_id,iso_year
0,1,1990
1,2,1991
2,3,1992
3,4,1993
4,5,1994
...,...,...
56,57,2046
57,58,2047
58,59,2048
59,60,2049


In [54]:
df_week

,calendar_weeks_id,iso_key,iso_week,calendar_years_fk
0,1,199001,1,1
1,2,199002,2,1
2,3,199003,3,1
3,4,199004,4,1
4,5,199005,5,1
...,...,...,...,...
3178,3179,205048,48,61
3179,3180,205049,49,61
3180,3181,205050,50,61
3181,3182,205051,51,61


In [55]:
df_day

,iso_day,calendar_weeks_fk
0,1990-01-01,1
1,1990-01-02,1
2,1990-01-03,1
3,1990-01-04,1
4,1990-01-05,1
...,...,...
22275,2050-12-27,3183
22276,2050-12-28,3183
22277,2050-12-29,3183
22278,2050-12-30,3183


In [1]:
import pandas as pd

In [2]:
agegroups_10y = { # agegroup, [number_observations, avg_age]
        "00-09": [10, 4.5],
        "10-19": [10, 14.5],
        "20-29": [10, 24.5],
        "30-39": [10, 34.5],
        "40-49": [10, 44.5],
        "50-59": [10, 54.5],
        "60-69": [10, 64.5],
        "70-79": [10, 74.5],
        "80+": [21, 90],
        "UNK": [0, 0],
}

In [11]:
df = pd.DataFrame.from_dict(agegroups_10y, orient="index", columns=["number_observations", "avg_age"]).reset_index(level=0).rename({"index": "agegroup"}, axis=1)
df

,agegroup,number_observations,avg_age
0,00-09,10,4.5
1,10-19,10,14.5
2,20-29,10,24.5
3,30-39,10,34.5
4,40-49,10,44.5
5,50-59,10,54.5
6,60-69,10,64.5
7,70-79,10,74.5
8,80+,21,90.0
9,UNK,0,0.0


In [31]:
import pandas as pd

df = pd.read_csv("countries.csv", encoding="utf8", na_filter = False).drop(["countries_id", "last_update", "unique_key"], axis=1)

df.head()

,country_en,country_de,latitude,longitude,iso_3166_1_alpha2,iso_3166_1_alpha3,iso_3166_1_numeric,nuts_0
0,Afghanistan,Afghanistan,33.939110,67.709953,af,afg,4,AF
1,Albania,Albanien,41.153332,20.168331,al,alb,8,AL
2,Antarctica,Antarktis (Sonderstatus durch Antarktisvertrag),-75.250973,-0.071389,aq,ata,10,AQ
3,Algeria,Algerien,28.033886,1.659626,dz,dza,12,DZ
4,American Samoa,Amerikanisch-Samoa,-14.270972,-170.132217,as,asm,16,AS
